In [1]:
import os

from tqdm import tqdm

from itertools import combinations

from dataset import read_atis
from utils import load_config

In [2]:
test = read_atis('test')

languages = load_config()['languages']

languages

['en', 'de', 'es', 'fr', 'ja', 'pt', 'zh_cn']

In [3]:
for language1, language2 in tqdm(combinations(languages, 2), total=21):
    with open(f'data/alignment/{language1}_{language2}.src', 'w') as fp:
        for idx, block in test.groupby('uuid'):
            sentence1 = block.loc[block['language'] == language1]['utterance'].values[0]
            sentence2 = block.loc[block['language'] == language2]['utterance'].values[0]

            print(sentence1, sentence2, sep=' ||| ', file=fp)

    language1, language2 = language2, language1
            
    with open(f'data/alignment/{language1}_{language2}.src', 'w') as fp:
        for idx, block in test.groupby('uuid'):
            sentence1 = block.loc[block['language'] == language1]['utterance'].values[0]
            sentence2 = block.loc[block['language'] == language2]['utterance'].values[0]

            print(sentence1, sentence2, sep=' ||| ', file=fp)

100%|██████████| 21/21 [00:25<00:00,  1.22s/it]


In [4]:
cmd = """
DATA_FILE=data/alignment/{}_{}.src
MODEL_NAME_OR_PATH=bert-base-multilingual-cased
OUTPUT_FILE=data/alignment/{}_{}.out

CUDA_VISIBLE_DEVICES=0 awesome-align \
    --output_file=$OUTPUT_FILE \
    --model_name_or_path=$MODEL_NAME_OR_PATH \
    --data_file=$DATA_FILE \
    --extraction 'softmax' \
    --batch_size 32 \
"""


for language1, language2 in tqdm(combinations(languages, 2), total=21):
    os.system(cmd.format(language1, language2, language1, language2))
    os.system(cmd.format(language2, language1, language2, language1))

100%|██████████| 21/21 [08:35<00:00, 24.54s/it]
